In [116]:
import numpy as np
import pandas as pd
from pandas import DataFrame
from sklearn.metrics import roc_auc_score,roc_curve
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import time
from matplotlib import pyplot as plt
#plot이 나오지 않을경우 한번 더 코드 실행

#seed 고정하기
np.random.seed(0)

#data이 저장 되어 있는 주소 입력하기
data=pd.read_csv('Final10.csv')
print(data.head())

data=data.drop(data.columns[0], axis='columns')
data=data.drop(data.columns[0], axis='columns')
print(data.head())
#범주형 변수를 카테고리화 하기
data.fr_yn=data.fr_yn.astype("category")
data.bldng_us=data.bldng_us.astype("category")
data.bldng_archtctr=data.bldng_archtctr.astype("category")
data.bldng_us_clssfctn=data.bldng_us_clssfctn.astype("category")
#data.lw_13101010=data.lw_13101010.astype("category")
#data.lw_13101110=data.lw_13101110.astype("category")
#data.lw_13101210=data.lw_13101210.astype("category")
#data.lw_13101211=data.lw_13101211.astype("category")
#data.lw_13101310=data.lw_13101310.astype("category")
#data.lw_13101410=data.lw_13101410.astype("category")
#data.lw_13111010=data.lw_13111010.astype("category")
#data.lw_13111110=data.lw_13111110.astype("category")
#data.lw_13121010=data.lw_13121010.astype("category")
#data.lw_13121011=data.lw_13121011.astype("category")
#data.lw_13131010=data.lw_13131010.astype("category")
#data.lw_13131110=data.lw_13131110.astype("category")
#data.lw_13141010=data.lw_13141010.astype("category")
#data.lw_13141011=data.lw_13141011.astype("category")
data.rgnl_ar_nm=data.rgnl_ar_nm.astype("category")
data.rgnl_ar_nm2=data.rgnl_ar_nm2.astype("category")
data.lnd_us_sttn_nm=data.lnd_us_sttn_nm.astype("category")
data.rd_sd_nm=data.rd_sd_nm.astype("category")
data.emd_nm=data.emd_nm.astype("category")
data.mlt_us_yn=data.mlt_us_yn.astype("category")
data.jmk=data.jmk.astype("category")
data.dt_of_athrztn_na=data.dt_of_athrztn_na.astype("category")
data.ttl_grnd_flr_na=data.ttl_grnd_flr_na.astype("category")
data.ttl_dwn_flr_na=data.ttl_dwn_flr_na.astype("category")
data.tmprtr_na=data.tmprtr_na.astype("category")
data.prcpttn_na=data.prcpttn_na.astype("category")
data.wnd_spd_na=data.wnd_spd_na.astype("category")
data.wnd_drctn_na=data.wnd_drctn_na.astype("category")
data.hmdt_na=data.hmdt_na.astype("category")
data.gas_engry_us_na=data.gas_engry_us_na.astype("category")
data.ele_engry_us_na=data.ele_engry_us_na.astype("category")
data.hm_cnt_na=data.hm_cnt_na.astype("category")
data.bldng_ar_prc_na=data.bldng_ar_prc_na.astype("category")
data.fr_mn_cnt_na=data.fr_mn_cnt_na.astype("category")
data.Year=data.Year.astype("category")
data.Month=data.Month.astype("category")
data.Day=data.Day.astype("category")
data.Hour=data.Hour.astype("category")
#data.lw_na=data.lw_na.astype("category")
data['fr_yn'] = (data['fr_yn'] == 'Y').astype(int)

#종속변수와 변인 나눠주기.
x=data.drop('fr_yn',axis=1)
y=data.fr_yn

print(x.head())
print(y.head())

#data1을 7:3으로 train 데이터, test 데이터로 나누기
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=.3)
train_data=lgb.Dataset(x_train,label=y_train)

#importance plot에 사용할 변수명 만들어주기.
#xname=['V1','V2','V3','V4','V5','V6','V7','T1','T2','W1','W2','W3','A1','A2','A3','A4','A5','A6','A7','A8','A9','A10','A11','A12']


#여러가지 값을 대입해보고 AUC값이 높게 나오는 파라미터를 찾았다.
param = {'boosting_type': 'gbdt','num_leaves':1000, 'objective':'binary',
'max_depth':-1,'learning_rate':.041,'max_bin':200,'importance_type':'gain'
 ,     }
param['metric'] = ['auc', 'binary_logloss']
num_boost_round = 200 #default 100
early_stopping_rounds = 50
nfold = 100
evals_result = {}
num_round = num_boost_round 
early_stopping_round=True
bagging_fraction=0.8
feature_fraction=0.9
scale_pos_weight=1.1


#모델링 소요시간 구하기
start = time.time()
lgbm=lgb.train(param,train_data,num_round)
print("time :", time.time() - start)


ypred=lgbm.predict(x_test)
#data1 predprob값 저장하기
predprob4_gbm=pd.DataFrame(ypred)

#data1 lightGBM AUC
print("data AUC")
print(roc_auc_score(y_test,ypred))



   Unnamed: 0          dt_of_fr fr_yn bldng_us bldng_archtctr  bldng_cnt  \
0           1  2017-10-20 05:54     Y     단독주택           블록구조          3   
1           2  2018-09-30 08:26     N      _na            _na          3   
2           3  2016-10-30 14:57     Y     공동주택       철근콘크리트구조          1   
3           4  2016-06-14 05:23     N     단독주택          일반목구조          2   
4           5  2018-04-22 05:38     N      _na            _na          2   

   bldng_ar   ttl_ar  lnd_ar  ttl_grnd_flr  ...  ele_engry_us_na lw_na  \
0     69.42    69.42     0.0             1  ...                0     1   
1     46.29    46.29     0.0             1  ...                1     1   
2    583.80  2516.76  1446.0             5  ...                0     1   
3     48.92    48.92     0.0             1  ...                0     1   
4      0.00     0.00     0.0             0  ...                1     1   

   hm_cnt_na  bldng_ar_prc_na  fr_mn_cnt_na  Year  Month  Day  Hour  Minute  
0          0        

In [129]:
ypred_yn = np.array([0 for i in range(len(ypred))])
for i in range(0,1):
    for j in range(0,len(ypred)):
        #type1 error와 type2 error의 비율이 거의 같게되는 cutoff값을 여러 값들을 대입해 보고 찾았다.
        if ypred[j]>0.017:
            ypred_yn[j]=1
        else:
            ypred_yn[j]=0

#data1 lightGBM importance plot
#print('Plotting feature importances...')
#ax = lgb.plot_importance(lgbm, max_num_features=20)
#plt.show()


#data1&data2 lightGBM Crosstab
print("Crosstab")
print(pd.crosstab(y_test,ypred_yn))

result1=(pd.crosstab(y_test,ypred_yn)[0][0])/((pd.crosstab(y_test,ypred_yn)[1][0])+(pd.crosstab(y_test,ypred_yn)[0][0]))

result2=(pd.crosstab(y_test,ypred_yn)[1][1])/((pd.crosstab(y_test,ypred_yn)[0][1])+(pd.crosstab(y_test,ypred_yn)[1][1]))
recall=result2
precision = ((pd.crosstab(y_test,ypred_yn)[0][0])+(pd.crosstab(y_test,ypred_yn)[1][1]))/((pd.crosstab(y_test,ypred_yn)[0][0])+(pd.crosstab(y_test,ypred_yn)[0][1])+(pd.crosstab(y_test,ypred_yn)[1][0])+(pd.crosstab(y_test,ypred_yn)[1][1]))
f1score=(precision*recall*2)/(precision+recall)
print("실제가 0인데 0으로 예측한 비율")     
print(result1)
print("실제가 1인데 1로 예측한 비율")     
print(result2)
print("f1score")
print(f1score)

Crosstab
col_0      0     1
fr_yn             
0      12434  4661
1        282  2453
실제가 0인데 0으로 예측한 비율
0.7273471775372916
실제가 1인데 1로 예측한 비율
0.8968921389396709
f1score
0.8173286980199573
